In [1]:
!pip install array-to-latex

  Created wheel for clipboard: filename=clipboard-0.0.4-py3-none-any.whl size=1850 sha256=05060c748807cb5b517b1eb2cf2a15296e255dd158adc5dc54d8782103701fca
  Stored in directory: c:\users\sambi\appdata\local\pip\cache\wheels\58\b3\11\d2a638f07f2b7f9210a1a1e8fd621005a836fa1dbf426a1553
  Created wheel for pyperclip: filename=pyperclip-1.8.2-py3-none-any.whl size=11136 sha256=4260d2bfaaa4fb9a3c564ab4341a56bb7b403ca22bac302e3333af03949adfc2
  Stored in directory: c:\users\sambi\appdata\local\pip\cache\wheels\0c\09\9e\49e21a6840ef7955b06d47394afef0058f0378c0914e48b8b8
Successfully built clipboard pyperclip


In [ ]:
import numpy as np

# Relaxation method
def relaxation_method(A, b, x0, tolerance=0.01, max_iterations=1000, omega=1.25):
    """
    Solve a system of linear equations using the Relaxation method.

    Arguments:
    A: 2D array representing the coefficients of the linear equations
    b: 1D array representing the constants on the right-hand side of the equations
    x0: 1D array representing the initial guess for the solution
    tolerance: Desired tolerance for the solution
    max_iterations: Maximum number of iterations allowed
    omega: Relaxation parameter (typically between 1 and 2)

    Returns:
    solution: 1D array representing the solution to the system of linear equations
    num_iterations: Number of iterations performed
    """
    n = len(b)
    solution = np.copy(x0)
    for iteration in range(max_iterations):
        old_solution = np.copy(solution)
        for i in range(n):
            summation = 0.0
            for j in range(n):
                if j != i:
                    summation=summation+A[i][j] * solution[j]
            solution[i] = (1 - omega) * old_solution[i] + (omega / A[i][i]) * (b[i] - summation)
        if np.linalg.norm(solution - old_solution) < tolerance:
            return solution, iteration + 1
    print("Warning: Maximum number of iterations reached without convergence.")
    return solution, max_iterations

# Solving by Relaxation method
solution, num_iterations = relaxation_method(A, b, x0)
print("Solution:", solution)
print("Number of iterations:", num_iterations)


In [11]:
import numpy as np
import array_to_latex as a2l
# Conjugate Gradient method
def conjugate_gradient(A, b, x0, tol=0.001, max_iter=1000):
    """
    Solve the linear system Ax = b using Conjugate Gradient method.

    Parameters:
        A (numpy.ndarray): Coefficient matrix.
        b (numpy.ndarray): Right-hand side vector.
        x0 (numpy.ndarray): Initial guess for the solution.
        tol (float): Tolerance for convergence.
        max_iter (int): Maximum number of iterations.

    Returns:
        x (numpy.ndarray): Solution vector.
        num_iter (int): Number of iterations performed.
    """
    float_formatter = "{:1.2f}".format
    np.set_printoptions(formatter={'float_kind':float_formatter})
    r = b - np.dot(A, x0)
    p = r
    print("p=")
    a2l.to_ltx(p, frmt = '{:1.2f}', arraytype = 'bmatrix')
    x = x0
    rsold = np.dot(r, r)
    print("|r|^2=",rsold)
    for i in range(max_iter):
        print("Iteration",i,":")
        print("r=")
        a2l.to_ltx(r, frmt = '{:1.2f}', arraytype = 'bmatrix')
        Ap = np.dot(A, p)
        print("Ap=")
        a2l.to_ltx(Ap, frmt = '{:1.2f}', arraytype = 'bmatrix')
        alpha = rsold / np.dot(p, Ap)
        print("alpha=",alpha)
        x = x + alpha * p
        print("x=")
        a2l.to_ltx(x, frmt = '{:1.2f}', arraytype = 'bmatrix')
        r = r - alpha * Ap
        print("r=")
        a2l.to_ltx(r, frmt = '{:1.2f}', arraytype = 'bmatrix')
        rsnew = np.dot(r, r)
        print("|r|^2=",rsnew)
        beta = (rsnew / rsold)
        print("beta",beta)
        if np.sqrt(rsnew) < tol:
            break
        p = r + beta * p
        print("new p=")
        a2l.to_ltx(p, frmt = '{:1.2f}', arraytype = 'bmatrix')
        rsold = rsnew
    return x, i+1

# Solving by Conjugate Gradient method
A = np.array([[1.00,0.50],[0.50,0.33]])
b = np.array([0.24,0.13])
x0 = np.array([0,0])
solution, iterations = conjugate_gradient(A, b, x0)
print("Solution:", solution)
print("Number of iterations:", iterations)


p=
\begin{bmatrix}
  0.24 &  0.13
\end{bmatrix}
|r|^2= 0.0745
Iteration 0 :
r=
\begin{bmatrix}
  0.24 &  0.13
\end{bmatrix}
Ap=
\begin{bmatrix}
  0.30 &  0.16
\end{bmatrix}
alpha= 0.7893872447736207
x=
\begin{bmatrix}
  0.19 &  0.10
\end{bmatrix}
r=
\begin{bmatrix}
 -0.00 &  0.00
\end{bmatrix}
|r|^2= 2.5671040149288605e-06
beta 3.445777201246793e-05
new p=
\begin{bmatrix}
 -0.00 &  0.00
\end{bmatrix}
Iteration 1 :
r=
\begin{bmatrix}
 -0.00 &  0.00
\end{bmatrix}
Ap=
\begin{bmatrix}
 -0.00 &  0.00
\end{bmatrix}
alpha= 15.835067114093954
x=
\begin{bmatrix}
  0.18 &  0.13
\end{bmatrix}
r=
\begin{bmatrix}
 -0.00 & -0.00
\end{bmatrix}
|r|^2= 3.722959880240703e-32
beta 1.450256732329513e-26
Solution: [0.18 0.13]
Number of iterations: 2
